In [ ]:
# to check, whether the results depend on shuffling
# test on the smallest set
%matplotlib inline
import matplotlib.pyplot as plt
import subprocess
import os
import random
import shutil
from collections import Counter
from collections import defaultdict
import statistics

DATASET_DIR = "../tests/input_files/test_100_25_25/"
TEMP_DIR = "temp"
DATASET_FILES = os.listdir(DATASET_DIR)
ANSWER = 25
STRNUM = 25
STRLEN = 100
os.mkdir(TEMP_DIR) if not os.path.isdir(TEMP_DIR) else None

In [ ]:
# create shuffled files
rep_files = []
der_to_orig = {}


print("Overall {} files in the dataset".format(len(DATASET_FILES)))
task_size = len(DATASET_FILES)
for num, to_check in enumerate(DATASET_FILES):
    # to_check = random.choice(DATASET_FILES)
    to_check_path = os.path.join(DATASET_DIR, to_check)
    replicate_min = {}  # number of replicate -> minimal answer
    
    shuffle_cmd_templ = "../shuffle_input.py " + to_check_path + " {} -o {} -c"
    for i in range(STRNUM):
        out_file = os.path.join(TEMP_DIR, "rep_{}_{}.txt".format(num, i))
        
        shuffle_cmd = shuffle_cmd_templ.format(i, out_file)
        rc = subprocess.call(shuffle_cmd, shell=True)
        if rc != 0:
            continue
        rep_files.append(out_file)
        der_to_orig[out_file] = to_check_path
    print("{}/{}".format(num + 1, task_size))

In [ ]:
# call CSP for each file
# a long stage
csp_cmd_templ = "../CSP {} {} -v 1"
print("Overall {} files to go".format(len(rep_files)))

orig_to_files_and_k = defaultdict(list)


for num, rep in enumerate(rep_files):
    # get the minimal answer for each
    csp_cmd = csp_cmd_templ.format(rep, ANSWER)
    csp_out = subprocess.check_output(csp_cmd, shell=True).decode("utf-8").split("\n")
    answ = csp_out[-3]
    if answ == "True":
        res_ = ANSWER
    else:
        max_cov_line = csp_out[-5].split()
        exp = int(max_cov_line[-1])
        real = int(max_cov_line[-3])
        diff = exp - real
        res_ = ANSWER + diff
    orig_ = der_to_orig[rep]
    replicate_min[num] = res_
    point = (rep, res_)
    orig_to_files_and_k[orig_].append(point)

In [ ]:
# visualize it
min_times = Counter()
for v in replicate_min.values():
    min_times[v] += 1
times_sort = sorted(min_times.keys())
for t in range(times_sort[0], times_sort[-1] + 1):
    app_ = min_times.get(t) if min_times.get(t) else 0
    print("{}:\n{}".format(t, "*" * app_))
# defenitely, algorithm is init string - dependent

In [ ]:
# maybe get some statistics?
# bunch of features
bests = []
reps = []
ks = []
l_nums = []
maxes = []
mins = []
deltas = []
aves = []
medians = []
stds = []
rep_best_line = defaultdict(set)
rep_line_p_sizes = defaultdict(list)
orig_done = set()
pat_sizes_arrs = []
pat_sizes_arr_means = []
pat_sizes_arr_medians = []
pat_arr_maxes = []
pat_arr_mins = []
pat_arr_stds = []


# analyze original file
for num, (k, v) in enumerate(orig_to_files_and_k.items()):
    v_sorted = sorted(v, key=lambda x: x[1])
    best_val = v_sorted[0][1]
    
    best_files = [x[0] for x in v_sorted if x[1] == best_val]
    for fle in best_files:
        best_file_info = fle.split("_")
        fle_num = int(best_file_info[1])
        line_num = int(best_file_info[2].split(".")[0])
        rep_best_line[fle_num].add(line_num)
    
    # work with original file
    f = open(k, "r")
    lines = [x[:-1] for x in f.readlines()]
    f.close()
    str_len = len(lines[0])
    str_num = len(lines)
    for i in range(str_len):
        column = [int(line[i]) for line in lines]
        zero_strings = [n for n, v in enumerate(column) if v == 0]
        one_strings = [n for n, v in enumerate(column) if v == 0]
        zero_pat_size = len(zero_strings)
        one_pat_size = len(one_strings)
        for elem in zero_strings:
            rep_line_p_sizes[(fle_num, elem)].append(zero_pat_size)
        for elem in one_strings:
            rep_line_p_sizes[(fle_num, elem)].append(one_pat_size)
    
# put everything in lists
for num, rep in enumerate(rep_files):
    # get the minimal answer for each
    fle_data = rep.split("_")
    fle_num = int(fle_data[1])

    l_num = int(fle_data[2].split(".")[0])
    l_nums.append(l_num)

    is_best = l_num in rep_best_line[fle_num]
    bests.append(is_best)

    pat_sizes_arr = rep_line_p_sizes[(fle_num, l_num)]
    pat_sizes_arrs.append(pat_sizes_arr)

    pat_sizes_arr_means.append(statistics.mean(pat_sizes_arr))
    pat_sizes_arr_medians.append(statistics.median(pat_sizes_arr))
    pat_arr_stds.append(statistics.stdev(pat_sizes_arr))
    
    
    pat_arr_maxes.append(max(pat_sizes_arr))
    pat_arr_mins.append(min(pat_sizes_arr))


    f = open(rep, "r")
    lines = [l[:-1] for l in f.readlines()]
    f.close()

    k = replicate_min[num]
    ks.append(k)

    zeros = sorted([line.count("0") for line in lines])
    max_zeros = zeros[-1]
    min_zeros = zeros[1]  # because in 0 -> 0
    maxes.append(max_zeros)
    mins.append(min_zeros)

    delta = max_zeros - min_zeros
    deltas.append(delta)

    aves.append(statistics.mean(zeros))
    medians.append(statistics.median(zeros))
    stds.append(statistics.stdev(zeros))

In [ ]:
# another visualisation?
fig = plt.figure(figsize=(30, 40))
rows = 4
cols = 3

mean_mean = statistics.mean(aves)
mean_std = statistics.mean(stds)
mean_delta = statistics.mean(deltas)
mean_pmax_size = statistics.mean(pat_arr_maxes)
mean_pmin_size = statistics.mean(pat_arr_mins)
mean_pat_arr_stdevs = statistics.mean(pat_arr_stds)
mean_pat_arr_mean = statistics.mean(pat_sizes_arr_means)

med_mean = statistics.median(aves)
med_std = statistics.median(stds)
med_delta = statistics.median(deltas)
med_pmax_size = statistics.median(pat_arr_maxes)
med_pmin_size = statistics.median(pat_arr_mins)
med_pat_arr_stdevs = statistics.median(pat_arr_stds)
med_pat_arr_mean = statistics.mean(pat_sizes_arr_means)

datasets = [maxes, mins, deltas, 
            aves, stds, pat_sizes_arr_means,
            pat_sizes_arr_medians, pat_arr_maxes, pat_arr_mins,
            pat_arr_stds, medians, bests]

names = ["Max_zeros", "Min_zeros", "Min_max_delta",
         "Mean", "Std", "pat_arr_mean",
         "pat_arr_med", "pat_arr_max", "pat_arr_mins",
         "pat_arr_stdevs", "median", "is_best"]

colors = ["red", "green", "blue",
          "magenta", "cyan", "#555555",
          "orange", "navy", "lightgreen",
          "red", "green", "blue"]

assert(len(datasets) == len(names) == len(colors))  # just in case

for num, data in enumerate(zip(datasets, names, colors), 1):
    ax = fig.add_subplot(rows, cols, num)
    ax.scatter(ks, data[0], s=20, color=data[2])
    # ax.scatter(bests, data[0], s=20, color=data[2])
    ax.set_xlabel("K")
    ax.set_ylabel(data[1])
    ax.grid(which="both")
    if data[1] == "Mean":
        ax.axhline(y=mean_mean, color='r', linestyle='-')
        ax.axhline(y=med_mean, color='g', linestyle='-')
    elif data[1] == "Std":
        ax.axhline(y=mean_std, color='r', linestyle='-')
        ax.axhline(y=med_std, color='g', linestyle='-')
    elif data[1] == "Min_max_delta":
        ax.axhline(y=mean_delta, color='r', linestyle='-')
        ax.axhline(y=med_delta, color='g', linestyle='-')
    elif data[1] == "pat_arr_max":
        ax.axhline(y=mean_pmax_size, color='r', linestyle='-')
        ax.axhline(y=med_pmax_size, color='g', linestyle='-')
    elif data[1] == "pat_arr_mins":
        ax.axhline(y=mean_pmin_size, color='r', linestyle='-')
        ax.axhline(y=med_pmin_size, color='g', linestyle='-')
    elif data[1] == "pat_arr_stdevs":
        ax.axhline(y=mean_pat_arr_stdevs, color='r', linestyle='-')
        ax.axhline(y=med_pat_arr_stdevs, color='g', linestyle='-')
    elif data[1] == "pat_arr_mean":
        ax.axhline(y=mean_pat_arr_mean, color='r', linestyle='-')
        ax.axhline(y=med_pat_arr_mean, color='g', linestyle='-')
plt.savefig("plots/shuffle_params.svg")
plt.show()
# somehow depends on max zeros and delta?

In [ ]:
best_ks, worse_ks = [], []
for elem in zip(bests, ks):
    if elem[0]:
        best_ks.append(elem[1])
    else:
        worse_ks.append(elem[1])

fig = plt.figure(figsize=(15, 8))
ax_1 = fig.add_subplot(1, 1, 1)
bins = 10
ax_1.hist(best_ks, color="blue", bins=bins, alpha=0.5)
ax_1.hist(worse_ks, color="red", bins=bins, alpha=0.5)

In [ ]:
shutil.rmtree(TEMP_DIR) if os.path.isdir(TEMP_DIR) else None